In [92]:
import pandas as pd
df = pd.read_csv(r'./richtig.csv', delimiter=";", error_bad_lines=False, engine ='python')

In [111]:
import re
import string
from germansentiment import SentimentModel
model = SentimentModel('mdraw/german-news-sentiment-bert')
terms = ['atheismus', 'christentum', 'islam', 'judentum', 'evangelismus', 'katholizismus']
filterIndex = []
#aus results.csv spalte: queryterm
chars = re.escape(string.punctuation)
for row in df.itertuples(index=True, name='Pandas'):
    cleanedString = row.tokensroot.strip("[]',").replace(r",", '').replace(r"'", '').split()
    for tokensroot in cleanedString:
        if tokensroot not in terms and row.Index not in filterIndex:
            filterIndex.append(row.Index)
            break;
            
df2 = df.drop(filterIndex)
len(df2)
df2['tokens_suggestions_cleaned_root_polarität'] = ''
for row in df2.itertuples(index=True, name='Pandas'):
    polarities = []
    cleanedString = row.tokens_suggestions_cleaned_root.strip("[]',").replace(r",", '').replace(r"'", '').split()
    for string in cleanedString:
        polarities.append(model.predict_sentiment(cleanedString))
    df2.at[row.Index, 'tokens_suggestions_cleaned_root_polarität'] = polarities

In [113]:
df2['tokens_suggestions_cleaned_root_polarität']

337      [[neutral, positive, negative, neutral, positi...
340      [[positive, neutral, positive, positive, neutr...
341      [[neutral, positive, neutral, neutral, positiv...
343      [[positive, neutral, positive, positive, neutr...
344      [[neutral, positive, negative, positive, posit...
                               ...                        
27007    [[neutral, positive, positive, negative, negat...
27022    [[neutral, positive, negative, negative, negat...
27027    [[negative, positive, negative, negative, neut...
27033    [[positive, negative, negative, positive, nega...
27040    [[negative, positive, negative, positive, posi...
Name: tokens_suggestions_cleaned_root_polarität, Length: 64, dtype: object

In [115]:
df2['tokens_suggestions_cleaned_root']

337      ['geschichte', 'feste', 'anhänger', 'religion'...
340      ['feste', 'verbreitung', 'fasten', 'referat', ...
341      ['geschichte', 'feste', 'verbreitung', 'religi...
343      ['feste', 'verbreitung', 'fasten', 'referat', ...
344      ['argumente', 'zitate', 'definition', 'anfänge...
                               ...                        
27007    ['geschichte', 'sprüche', 'zitate', 'makhachev...
27022    ['geschichte', 'feste', 'steckbrief', 'english...
27027    ['lamm', 'china', 'english', 'definition', 'be...
27033    ['deutschland', 'glaube', 'definition', 'in', ...
27040    ['symbol', 'prayer', 'definition', 'for', '360...
Name: tokens_suggestions_cleaned_root, Length: 64, dtype: object

In [116]:
df2.to_csv('richtig2.csv', sep=';', index=False, encoding='utf-8')
df_from_csv2 = pd.read_csv("richtig2.csv", sep=';', encoding='utf-8')